In [1]:
#get the dependencies that will be needed
import pandas as pd

In [2]:
#Load the raw data into the dataframes
school_data_raw = pd.read_csv("Resources/schools_complete.csv")
student_data_raw = pd.read_csv("Resources/students_complete.csv")

In [3]:
#Merged the dataframes together
school_data_complete = pd.merge(student_data_raw, school_data_raw, how="left", on=["school_name", "school_name"])

In [4]:
#Taking a look at the data. 
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [5]:
#using .unique() will give us the number of highschools in our data frame since there should not be any duplicate schools
number_of_schools = len(school_data_complete['school_name'].unique())

In [6]:
#For students we cannot assume there will not be duplicate names, but luckily each row is a unique student
number_of_students = len(school_data_complete['student_name'])

In [7]:
#To find the sum of the budget, we can just take the total from the budget column in the school dataframe
sum_budget = school_data_raw['budget'].sum()

In [14]:
#Finding the average score is simple using the .mean() function on the respective columns
average_math_score = school_data_complete["math_score"].mean()
average_reading_score = school_data_complete["reading_score"].mean()
average_overall_score = (average_math_score+average_reading_score)/2

In [9]:
#Now we need to find the percentage of passing students which is the total number of students with scores >70 / total students
passing_grade = 70
#First for reading
pass_reading = len(school_data_complete[school_data_complete['reading_score'] >= passing_grade])
pass_reading_per = pass_reading/number_of_students * 100
#Then for math
pass_math = len(school_data_complete[school_data_complete['math_score'] >= passing_grade])
pass_math_per = pass_math/number_of_students * 100
#Now it wants the average passing to be the average of the two averages
pass_overall_per = (pass_math_per + pass_reading_per)/2

In [16]:
#Next I am going to add all of these data points int a dictionary that will be turned into a dataframe
district_summary_dict = {"Total Schools":number_of_schools,"Total Students":number_of_students,"Total Budget":sum_budget,
                         "Average Math Score":average_math_score,"Average Reading Score":average_reading_score,
                         "% Passing Math":pass_math_per,"% Passing Reading":pass_reading_per,
                         "% Overall Passing Rate":average_overall_score}
district_summary_df = pd.DataFrame(district_summary_dict, index=[0])
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.431606


In [ ]:
#For the School summary section I can use the raw school data as a starting point, i just need to make the School name the index
school

In [28]:
school_data_raw.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [17]:
student_data_raw.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84
